In [1]:
"ok"

'ok'

In [182]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
import pinecone
from dotenv import load_dotenv

In [183]:
load_dotenv()

True

In [184]:
import os
from pinecone import Pinecone, ServerlessSpec

# Load key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found. Please set it first!")

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)


In [ ]:
index_name = "all-in-one-agent"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        )
    )



In [186]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
base_dir = "data" 


In [188]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

all_chunks = []


In [ ]:
for domain in os.listdir(base_dir):
    domain_path = os.path.join(base_dir, domain)

    if os.path.isdir(domain_path):
        for file in os.listdir(domain_path):
            if file.endswith(".pdf"):
                file_path = os.path.join(domain_path, file)
                print(f"📄 Processing {file_path} (domain: {domain})")

                loader = PyPDFLoader(file_path)
                docs = loader.load()
                chunks = splitter.split_documents(docs)

                for chunk in chunks:
                    chunk.metadata["domain"] = domain
                    chunk.metadata["source_file"] = file

                all_chunks.extend(chunks)

📄 Processing data/wellness/14.DavidWerner-WhereThereIsNoDoctor.pdf (domain: wellness)
📄 Processing data/finance/Personal Finance for Dummies.pdf (domain: finance)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong 

📄 Processing data/finance/Personal Financial Planning - GRAD - 2021 August.pdf (domain: finance)
📄 Processing data/home_diy/pdfcoffee.com_diy-and-home-improvements-handbook-pdf-free.pdf (domain: home_diy)
📄 Processing data/home_diy/make-yourself-at-home-diy-guide-070618.pdf (domain: home_diy)
📄 Processing data/home_diy/the-complete-idiots-guide-to-simple-home-repair.pdf (domain: home_diy)
📄 Processing data/home_diy/The Complete Do-it-Yourself Manual.pdf (domain: home_diy)
📄 Processing data/travel/The World.pdf (domain: travel)
📄 Processing data/travel/Colette Worldwide Travel Guide 2021-2023.pdf (domain: travel)


In [190]:
from langchain_pinecone import PineconeVectorStore


In [191]:
print(f"🚀 Uploading {len(all_chunks)} chunks to Pinecone...")
vectorstore = PineconeVectorStore.from_documents(
    documents=all_chunks,
    embedding=embeddings,
    index_name=index_name
)


🚀 Uploading 9487 chunks to Pinecone...


In [192]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [47]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [48]:
from langchain_openai import ChatOpenAI


In [51]:
load_dotenv()

True

In [52]:
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")


In [53]:
llm = ChatOpenAI(
            model="openai/gpt-oss-20b:free",
            temperature=0,
            openai_api_key=OPENROUTER_KEY,
            base_url="https://openrouter.ai/api/v1",
            default_headers={"HTTP-Referer": "http://localhost:8000", "X-Title": "AI Data Analyst"}
        )

In [54]:
from langchain.chains import RetrievalQA


In [55]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

In [56]:
query = "Give me some hobby ideas that are popular in India."
result = qa.run(query)

/var/folders/vr/k31rtn_168v7kvg7k8xq_v540000gn/T/ipykernel_65704/538902407.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


In [157]:
llm = ChatOpenAI(
            model="openai/gpt-oss-20b:free",
            temperature=0,
            openai_api_key=OPENROUTER_KEY,
            base_url="https://openrouter.ai/api/v1",
            default_headers={"HTTP-Referer": "http://localhost:8000", "X-Title": "AI Data Analyst"}
        )

In [ ]:
def classify_domain_with_llm(query):
    prompt = f"""
You are a smart assistant that classifies user questions into one of the following domains:
1. Finance & Budgeting
2. Travel & Local Guide
3. Home & DIY
4. Hobbies & Skills
5. Health & Wellness

Classify the following question into **only one domain**. Respond with the domain exactly as listed above.

Question: "{query}"
"""
    response = llm.invoke(prompt)
    return response.content.strip()

query = "What are some tips for planning a budget-friendly vacation?"
domain = classify_domain_with_llm(query)
print(domain)  

Travel & Local Guide


In [ ]:
vectorstore = PineconeVectorStore.from_existing_index(index_name, embeddings)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4},
    filter={"domain": "tavel"}
)

docs = retriever.get_relevant_documents(query)


In [195]:
print("Retrieved docs:", len(docs))
for d in docs:
    print(d.page_content[:300])
    print(d.metadata)


Retrieved docs: 4
deal of money. Many movies, theaters, museums, and restaurants offer 
discount prices on certain days and times.
Cultivate some interests and hobbies that are free or low-cost. Visiting with 
friends, hiking, reading, and playing sports can be good for your finances as 
well as your health.
Vacation
{'author': 'Tyson, Eric.', 'creationdate': '2009-10-01T00:15:41-04:00', 'creator': 'Adobe InDesign CS4 (6.0.2)', 'domain': 'finance', 'moddate': '2013-10-14T22:02:30-04:00', 'page': 139.0, 'page_label': '140', 'producer': 'Acrobat Distiller 6.0.1 (Windows)', 'source': 'data/finance/Personal Finance for Dummies.pdf', 'source_file': 'Personal Finance for Dummies.pdf', 'title': 'Personal Finance for Dummies', 'total_pages': 483.0}
deal of money. Many movies, theaters, museums, and restaurants offer 
discount prices on certain days and times.
Cultivate some interests and hobbies that are free or low-cost. Visiting with 
friends, hiking, reading, and playing sports can be good 

In [196]:
context = "\n\n".join([
    f"[{d.metadata['domain']} | {d.metadata['source_file']} | page {d.metadata.get('page', '?')}]\n{d.page_content}"
    for d in docs
])


In [197]:
from langchain.prompts import PromptTemplate

template = """
You are a specialized assistant that answers questions STRICTLY based on the provided documents and chat history.

*** CRITICAL RULES ***
1.  **Grounding:** Your entire response MUST be based SOLELY on the information within the "Retrieved Documents" section below.
2.  **Domain Extraction:** If the question involves domains, you MUST return ONLY the unique values of the field "domain" from the retrieved documents.
      - Each domain should be listed on a new line.
      - Do not add explanations, summaries, or extra text.
3.  **No External Knowledge:** DO NOT use any external knowledge, personal opinions, or information you were trained on. Your knowledge is limited to the documents provided.
4.  **Handling Missing Information:** If the documents and history do not contain enough information to answer the question, you MUST explicitly state: "The provided materials do not contain enough information to answer this question."
5.  **Citation:** You must cite the source for every piece of information you provide, using the format [source_file | page]. 
      - If only domains are asked, citations are NOT required.
6.  **Conciseness & Formatting:** Keep the response under 400 words and use Markdown for clarity (headings, bullet points, bold text).


Documents:
{context}

Question:
{question}

Answer in a clear, concise, and practical way (bullet points if suitable), and include the exact sources.
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=template)


In [ ]:
from langchain_openai import ChatOpenAI  


final_prompt = prompt.format(context=context, question=query)
response = llm.invoke(final_prompt)

print("🤖 LLM Answer:\n", response.content)


🤖 LLM Answer:
 **Budget‑friendly vacation tips**

- Avoid borrowing on credit cards for travel expenses; vacations are not investments.  
  *[finance | Personal Finance for Dummies.pdf | page 139.0]*  

- Opt for shorter trips closer to home, such as state or national parks nearby.  
  *[finance Finance for Dummies.pdf | page 139.0]*  

- Plan a “stay‑cation” by visiting local sites within 200 miles that you’ve always wanted to see.  
  *[finance | Personal Finance for Dummies.pdf | page 139.0]*  

- Take advantage of discount days and times at movies, theaters, museums, and restaurants to save on entertainment.  
  *[finance | Personal Finance for Dummies.pdf | page 139.0]*
